In [ ]:
!apt-get update

In [ ]:
!apt-get install -y build-essential

In [ ]:
!python -m pip install -q torch==1.10.2 torchvision==0.11.3
!python -m pip install -q transformers datasets
!python -m pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.10/index.html
!python -m pip install -q seqeval sentencepiece

In [ ]:
!python -m pip install --upgrade -q numpy

In [6]:
import torch
torch.__version__

'1.10.2+cu102'

In [7]:
import torchvision
torchvision.__version__

'0.11.3+cu102'

In [8]:
import io
import re
import math
import base64

from PIL import Image

import torch
import numpy as np
import transformers

from torch.utils.data import Dataset, DataLoader

In [9]:
base64_image = "..."

In [10]:
blocks = [
  {
    "Text": "Sokos",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08576438575983047,
        "Height": 0.018576089292764664,
        "Left": 0.23532548546791077,
        "Top": 0.17296446859836578
      },
      "Polygon": [
        {
          "X": 0.2353723645210266,
          "Y": 0.17296446859836578
        },
        {
          "X": 0.32108986377716064,
          "Y": 0.17334072291851044
        },
        {
          "X": 0.32107922434806824,
          "Y": 0.19154055416584015
        },
        {
          "X": 0.23532548546791077,
          "Y": 0.19116665422916412
        }
      ]
    }
  },
  {
    "Text": "Helsinki",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.13503558933734894,
        "Height": 0.019257284700870514,
        "Left": 0.33889734745025635,
        "Top": 0.17324066162109375
      },
      "Polygon": [
        {
          "X": 0.33890053629875183,
          "Y": 0.17324066162109375
        },
        {
          "X": 0.4738776385784149,
          "Y": 0.17383317649364471
        },
        {
          "X": 0.4739329516887665,
          "Y": 0.19249793887138367
        },
        {
          "X": 0.33889734745025635,
          "Y": 0.19190922379493713
        }
      ]
    }
  },
  {
    "Text": "p.",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.02878890186548233,
        "Height": 0.016177132725715637,
        "Left": 0.4949924051761627,
        "Top": 0.17837125062942505
      },
      "Polygon": [
        {
          "X": 0.4949924051761627,
          "Y": 0.17837125062942505
        },
        {
          "X": 0.5237151980400085,
          "Y": 0.17849713563919067
        },
        {
          "X": 0.5237812995910645,
          "Y": 0.19454838335514069
        },
        {
          "X": 0.49504783749580383,
          "Y": 0.19442319869995117
        }
      ]
    }
  },
  {
    "Text": "010",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.051018454134464264,
        "Height": 0.01782812923192978,
        "Left": 0.5464516878128052,
        "Top": 0.17457540333271027
      },
      "Polygon": [
        {
          "X": 0.5464516878128052,
          "Y": 0.17457540333271027
        },
        {
          "X": 0.5973674654960632,
          "Y": 0.1747988760471344
        },
        {
          "X": 0.5974701046943665,
          "Y": 0.19240352511405945
        },
        {
          "X": 0.5465335249900818,
          "Y": 0.19218140840530396
        }
      ]
    }
  },
  {
    "Text": "766",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.04956797882914543,
        "Height": 0.01796928234398365,
        "Left": 0.6169029474258423,
        "Top": 0.17441140115261078
      },
      "Polygon": [
        {
          "X": 0.6169029474258423,
          "Y": 0.17441140115261078
        },
        {
          "X": 0.6663389205932617,
          "Y": 0.17462842166423798
        },
        {
          "X": 0.6664708852767944,
          "Y": 0.19238068163394928
        },
        {
          "X": 0.6170145273208618,
          "Y": 0.19216498732566833
        }
      ]
    }
  },
  {
    "Text": "5100",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06812912970781326,
        "Height": 0.01778312772512436,
        "Left": 0.6851441264152527,
        "Top": 0.17472928762435913
      },
      "Polygon": [
        {
          "X": 0.6851441264152527,
          "Y": 0.17472928762435913
        },
        {
          "X": 0.7531080842018127,
          "Y": 0.17502763867378235
        },
        {
          "X": 0.7532732486724854,
          "Y": 0.1925124228000641
        },
        {
          "X": 0.6852817535400391,
          "Y": 0.19221585988998413
        }
      ]
    }
  },
  {
    "Text": "Hok-Elanto",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.17248329520225525,
        "Height": 0.02018127590417862,
        "Left": 0.16600097715854645,
        "Top": 0.19742785394191742
      },
      "Polygon": [
        {
          "X": 0.16608227789402008,
          "Y": 0.19742785394191742
        },
        {
          "X": 0.3384842574596405,
          "Y": 0.1981777399778366
        },
        {
          "X": 0.33848074078559875,
          "Y": 0.21760912239551544
        },
        {
          "X": 0.16600097715854645,
          "Y": 0.21686430275440216
        }
      ]
    }
  },
  {
    "Text": "Liiketoiminta",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.22182364761829376,
        "Height": 0.01922808773815632,
        "Left": 0.35760200023651123,
        "Top": 0.19882258772850037
      },
      "Polygon": [
        {
          "X": 0.35760200023651123,
          "Y": 0.19882258772850037
        },
        {
          "X": 0.579326868057251,
          "Y": 0.19978679716587067
        },
        {
          "X": 0.5794256329536438,
          "Y": 0.21805067360401154
        },
        {
          "X": 0.35760679841041565,
          "Y": 0.2170925736427307
        }
      ]
    }
  },
  {
    "Text": "Oy,",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.04590409994125366,
        "Height": 0.020368754863739014,
        "Left": 0.5982572436332703,
        "Top": 0.1997005045413971
      },
      "Polygon": [
        {
          "X": 0.5982572436332703,
          "Y": 0.1997005045413971
        },
        {
          "X": 0.6440219879150391,
          "Y": 0.19989953935146332
        },
        {
          "X": 0.6441613435745239,
          "Y": 0.2200692594051361
        },
        {
          "X": 0.5983752012252808,
          "Y": 0.2198716253042221
        }
      ]
    }
  },
  {
    "Text": "1837957-3",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.1505274623632431,
        "Height": 0.018949931487441063,
        "Left": 0.6879990100860596,
        "Top": 0.19908931851387024
      },
      "Polygon": [
        {
          "X": 0.6879990100860596,
          "Y": 0.19908931851387024
        },
        {
          "X": 0.8383175134658813,
          "Y": 0.19974328577518463
        },
        {
          "X": 0.8385264873504639,
          "Y": 0.21803924441337585
        },
        {
          "X": 0.6881441473960876,
          "Y": 0.21738941967487335
        }
      ]
    }
  },
  {
    "Text": "33",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.03363602235913277,
        "Height": 0.01813066005706787,
        "Left": 0.1483735889196396,
        "Top": 0.24758388102054596
      },
      "Polygon": [
        {
          "X": 0.14845609664916992,
          "Y": 0.24758388102054596
        },
        {
          "X": 0.18200962245464325,
          "Y": 0.24772711098194122
        },
        {
          "X": 0.18194110691547394,
          "Y": 0.26571452617645264
        },
        {
          "X": 0.1483735889196396,
          "Y": 0.2655722200870514
        }
      ]
    }
  },
  {
    "Text": "K33",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.05232471972703934,
        "Height": 0.01820763573050499,
        "Left": 0.20005804300308228,
        "Top": 0.24795086681842804
      },
      "Polygon": [
        {
          "X": 0.2001190036535263,
          "Y": 0.24795086681842804
        },
        {
          "X": 0.252382755279541,
          "Y": 0.24817395210266113
        },
        {
          "X": 0.25234362483024597,
          "Y": 0.26615849137306213
        },
        {
          "X": 0.20005804300308228,
          "Y": 0.26593682169914246
        }
      ]
    }
  },
  {
    "Text": "M000280/2960",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.2068665623664856,
        "Height": 0.021833863109350204,
        "Left": 0.26991063356399536,
        "Top": 0.24719324707984924
      },
      "Polygon": [
        {
          "X": 0.26994770765304565,
          "Y": 0.24719324707984924
        },
        {
          "X": 0.4767138361930847,
          "Y": 0.2480761706829071
        },
        {
          "X": 0.47677716612815857,
          "Y": 0.26902711391448975
        },
        {
          "X": 0.26991063356399536,
          "Y": 0.2681507468223572
        }
      ]
    }
  },
  {
    "Text": "21:04",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08655162155628204,
        "Height": 0.01867084577679634,
        "Left": 0.5290255546569824,
        "Top": 0.24873094260692596
      },
      "Polygon": [
        {
          "X": 0.5290255546569824,
          "Y": 0.24873094260692596
        },
        {
          "X": 0.6154629588127136,
          "Y": 0.2490999847650528
        },
        {
          "X": 0.6155771613121033,
          "Y": 0.267401784658432
        },
        {
          "X": 0.5291031002998352,
          "Y": 0.2670351266860962
        }
      ]
    }
  },
  {
    "Text": "29-11-2021",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.16862687468528748,
        "Height": 0.01892191544175148,
        "Left": 0.6666434407234192,
        "Top": 0.2486967295408249
      },
      "Polygon": [
        {
          "X": 0.6666434407234192,
          "Y": 0.2486967295408249
        },
        {
          "X": 0.8350639939308167,
          "Y": 0.24941596388816833
        },
        {
          "X": 0.8352702856063843,
          "Y": 0.26761865615844727
        },
        {
          "X": 0.6667786240577698,
          "Y": 0.2669040560722351
        }
      ]
    }
  },
  {
    "Text": "M&S",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.05230802297592163,
        "Height": 0.01818181201815605,
        "Left": 0.14690399169921875,
        "Top": 0.2969360649585724
      },
      "Polygon": [
        {
          "X": 0.14698690176010132,
          "Y": 0.2969360649585724
        },
        {
          "X": 0.19921201467514038,
          "Y": 0.29715484380722046
        },
        {
          "X": 0.19915083050727844,
          "Y": 0.31511786580085754
        },
        {
          "X": 0.14690399169921875,
          "Y": 0.3149005174636841
        }
      ]
    }
  },
  {
    "Text": "PERUNA",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08476875722408295,
        "Height": 0.018732959404587746,
        "Left": 0.21828718483448029,
        "Top": 0.29746052622795105
      },
      "Polygon": [
        {
          "X": 0.2183416485786438,
          "Y": 0.29746052622795105
        },
        {
          "X": 0.30305594205856323,
          "Y": 0.29781538248062134
        },
        {
          "X": 0.30303755402565,
          "Y": 0.31619349122047424
        },
        {
          "X": 0.21828718483448029,
          "Y": 0.3158409893512726
        }
      ]
    }
  },
  {
    "Text": "3-PA",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.0683021992444992,
        "Height": 0.018624847754836082,
        "Left": 0.511275053024292,
        "Top": 0.29745474457740784
      },
      "Polygon": [
        {
          "X": 0.511275053024292,
          "Y": 0.29745474457740784
        },
        {
          "X": 0.5794782638549805,
          "Y": 0.2977405786514282
        },
        {
          "X": 0.5795772671699524,
          "Y": 0.31607961654663086
        },
        {
          "X": 0.5113451480865479,
          "Y": 0.3157956600189209
        }
      ]
    }
  },
  {
    "Text": "14.95",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08249912410974503,
        "Height": 0.018575675785541534,
        "Left": 0.7387295365333557,
        "Top": 0.2976139783859253
      },
      "Polygon": [
        {
          "X": 0.7387295365333557,
          "Y": 0.2976139783859253
        },
        {
          "X": 0.8210282325744629,
          "Y": 0.29795899987220764
        },
        {
          "X": 0.8212286829948425,
          "Y": 0.3161896765232086
        },
        {
          "X": 0.7388952374458313,
          "Y": 0.31584692001342773
        }
      ]
    }
  },
  {
    "Text": "22270177",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.13741809129714966,
        "Height": 0.0191947054117918,
        "Left": 0.18236112594604492,
        "Top": 0.3214845359325409
      },
      "Polygon": [
        {
          "X": 0.18243177235126495,
          "Y": 0.3214845359325409
        },
        {
          "X": 0.3197792172431946,
          "Y": 0.32205453515052795
        },
        {
          "X": 0.3197678029537201,
          "Y": 0.34067925810813904
        },
        {
          "X": 0.18236112594604492,
          "Y": 0.34011316299438477
        }
      ]
    }
  },
  {
    "Text": "N.ALUSH.LAHKEETON",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.2974970042705536,
        "Height": 0.020031772553920746,
        "Left": 0.4222498834133148,
        "Top": 0.32109785079956055
      },
      "Polygon": [
        {
          "X": 0.4222498834133148,
          "Y": 0.32109785079956055
        },
        {
          "X": 0.7195844650268555,
          "Y": 0.3223324120044708
        },
        {
          "X": 0.7197468876838684,
          "Y": 0.3411296010017395
        },
        {
          "X": 0.42228296399116516,
          "Y": 0.33990350365638733
        }
      ]
    }
  },
  {
    "Text": "YHTEENSA",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.13779759407043457,
        "Height": 0.019038641825318336,
        "Left": 0.1480609029531479,
        "Top": 0.3702522814273834
      },
      "Polygon": [
        {
          "X": 0.14814555644989014,
          "Y": 0.3702522814273834
        },
        {
          "X": 0.28585848212242126,
          "Y": 0.37081292271614075
        },
        {
          "X": 0.2858326733112335,
          "Y": 0.3892909288406372
        },
        {
          "X": 0.1480609029531479,
          "Y": 0.3887341320514679
        }
      ]
    }
  },
  {
    "Text": "14.95",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08213834464550018,
        "Height": 0.01841864362359047,
        "Left": 0.7393952012062073,
        "Top": 0.36998075246810913
      },
      "Polygon": [
        {
          "X": 0.7393952012062073,
          "Y": 0.36998075246810913
        },
        {
          "X": 0.821334958076477,
          "Y": 0.37031468749046326
        },
        {
          "X": 0.8215335607528687,
          "Y": 0.3883993923664093
        },
        {
          "X": 0.7395595908164978,
          "Y": 0.38806769251823425
        }
      ]
    }
  },
  {
    "Text": "KORTTITAPAHTUMA",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.25901398062705994,
        "Height": 0.021288922056555748,
        "Left": 0.14762595295906067,
        "Top": 0.3929488956928253
      },
      "Polygon": [
        {
          "X": 0.1477188766002655,
          "Y": 0.3929488956928253
        },
        {
          "X": 0.40661171078681946,
          "Y": 0.39399340748786926
        },
        {
          "X": 0.4066399335861206,
          "Y": 0.4142377972602844
        },
        {
          "X": 0.14762595295906067,
          "Y": 0.413201242685318
        }
      ]
    }
  },
  {
    "Text": "Kortti:",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.11765118688344955,
        "Height": 0.019207457080483437,
        "Left": 0.14572523534297943,
        "Top": 0.44314196705818176
      },
      "Polygon": [
        {
          "X": 0.14581196010112762,
          "Y": 0.44314196705818176
        },
        {
          "X": 0.2633764147758484,
          "Y": 0.44360679388046265
        },
        {
          "X": 0.263340562582016,
          "Y": 0.46234944462776184
        },
        {
          "X": 0.14572523534297943,
          "Y": 0.46188798546791077
        }
      ]
    }
  },
  {
    "Text": "VISA",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06928588449954987,
        "Height": 0.018129222095012665,
        "Left": 0.5975269675254822,
        "Top": 0.4427568018436432
      },
      "Polygon": [
        {
          "X": 0.5975269675254822,
          "Y": 0.4427568018436432
        },
        {
          "X": 0.6666808128356934,
          "Y": 0.443030446767807
        },
        {
          "X": 0.6668128371238708,
          "Y": 0.46088600158691406
        },
        {
          "X": 0.597630500793457,
          "Y": 0.46061423420906067
        }
      ]
    }
  },
  {
    "Text": "ELECTRON",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.13803930580615997,
        "Height": 0.019017506390810013,
        "Left": 0.6859582662582397,
        "Top": 0.44192010164260864
      },
      "Polygon": [
        {
          "X": 0.6859582662582397,
          "Y": 0.44192010164260864
        },
        {
          "X": 0.8237940073013306,
          "Y": 0.44246581196784973
        },
        {
          "X": 0.8239975571632385,
          "Y": 0.46093758940696716
        },
        {
          "X": 0.686103105545044,
          "Y": 0.46039578318595886
        }
      ]
    }
  },
  {
    "Text": "****",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06945742666721344,
        "Height": 0.014280347153544426,
        "Left": 0.441270649433136,
        "Top": 0.46882787346839905
      },
      "Polygon": [
        {
          "X": 0.441270649433136,
          "Y": 0.46882787346839905
        },
        {
          "X": 0.5106749534606934,
          "Y": 0.4690995216369629
        },
        {
          "X": 0.5107280611991882,
          "Y": 0.4831082224845886
        },
        {
          "X": 0.4413013458251953,
          "Y": 0.4828380346298218
        }
      ]
    }
  },
  {
    "Text": "****",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06948591768741608,
        "Height": 0.014259842224419117,
        "Left": 0.5280297994613647,
        "Top": 0.46861532330513
      },
      "Polygon": [
        {
          "X": 0.5280297994613647,
          "Y": 0.46861532330513
        },
        {
          "X": 0.5974346995353699,
          "Y": 0.468887060880661
        },
        {
          "X": 0.5975157618522644,
          "Y": 0.48287516832351685
        },
        {
          "X": 0.5280884504318237,
          "Y": 0.4826049208641052
        }
      ]
    }
  },
  {
    "Text": "****",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.0681244432926178,
        "Height": 0.015034540556371212,
        "Left": 0.6158211827278137,
        "Top": 0.46821120381355286
      },
      "Polygon": [
        {
          "X": 0.6158211827278137,
          "Y": 0.46821120381355286
        },
        {
          "X": 0.6838306784629822,
          "Y": 0.4684775471687317
        },
        {
          "X": 0.6839456558227539,
          "Y": 0.48324573040008545
        },
        {
          "X": 0.6159130334854126,
          "Y": 0.4829809069633484
        }
      ]
    }
  },
  {
    "Text": "9701",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06557167321443558,
        "Height": 0.018092777580022812,
        "Left": 0.7024264931678772,
        "Top": 0.4665667712688446
      },
      "Polygon": [
        {
          "X": 0.7024264931678772,
          "Y": 0.4665667712688446
        },
        {
          "X": 0.7678247690200806,
          "Y": 0.46682310104370117
        },
        {
          "X": 0.7679981589317322,
          "Y": 0.4846595525741577
        },
        {
          "X": 0.7025730609893799,
          "Y": 0.484404981136322
        }
      ]
    }
  },
  {
    "Text": "LN",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.03291938081383705,
        "Height": 0.018057379871606827,
        "Left": 0.7912147641181946,
        "Top": 0.4662954807281494
      },
      "Polygon": [
        {
          "X": 0.7912147641181946,
          "Y": 0.4662954807281494
        },
        {
          "X": 0.8239365816116333,
          "Y": 0.46642374992370605
        },
        {
          "X": 0.8241341710090637,
          "Y": 0.48435285687446594
        },
        {
          "X": 0.791398823261261,
          "Y": 0.48422545194625854
        }
      ]
    }
  },
  {
    "Text": "Sovellus:",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.15237317979335785,
        "Height": 0.020334426313638687,
        "Left": 0.14437459409236908,
        "Top": 0.49081340432167053
      },
      "Polygon": [
        {
          "X": 0.14446648955345154,
          "Y": 0.49081340432167053
        },
        {
          "X": 0.29674777388572693,
          "Y": 0.4914037883281708
        },
        {
          "X": 0.29672521352767944,
          "Y": 0.5111477971076965
        },
        {
          "X": 0.14437459409236908,
          "Y": 0.510562002658844
        }
      ]
    }
  },
  {
    "Text": "A0000000032010",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.2453007698059082,
        "Height": 0.020815270021557808,
        "Left": 0.5795959830284119,
        "Top": 0.48931723833084106
      },
      "Polygon": [
        {
          "X": 0.5795959830284119,
          "Y": 0.48931723833084106
        },
        {
          "X": 0.8246777057647705,
          "Y": 0.49026864767074585
        },
        {
          "X": 0.8248967528343201,
          "Y": 0.5101324915885925
        },
        {
          "X": 0.5797029137611389,
          "Y": 0.5091885328292847
        }
      ]
    }
  },
  {
    "Text": "Tap.nro/Varmennus:",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.30428245663642883,
        "Height": 0.022438591346144676,
        "Left": 0.14737844467163086,
        "Top": 0.5157426595687866
      },
      "Polygon": [
        {
          "X": 0.147475928068161,
          "Y": 0.5157426595687866
        },
        {
          "X": 0.4516092836856842,
          "Y": 0.5169095396995544
        },
        {
          "X": 0.4516609013080597,
          "Y": 0.5381813049316406
        },
        {
          "X": 0.14737844467163086,
          "Y": 0.5370242595672607
        }
      ]
    }
  },
  {
    "Text": "06196/514778",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.2068404108285904,
        "Height": 0.022098246961832047,
        "Left": 0.6168034076690674,
        "Top": 0.5139145255088806
      },
      "Polygon": [
        {
          "X": 0.6168034076690674,
          "Y": 0.5139145255088806
        },
        {
          "X": 0.8234096169471741,
          "Y": 0.514708399772644
        },
        {
          "X": 0.8236438632011414,
          "Y": 0.5360127687454224
        },
        {
          "X": 0.6169362664222717,
          "Y": 0.5352255702018738
        }
      ]
    }
  },
  {
    "Text": "Yritys/Ala:",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.18647347390651703,
        "Height": 0.02286474220454693,
        "Left": 0.1462840586900711,
        "Top": 0.5395888686180115
      },
      "Polygon": [
        {
          "X": 0.14638608694076538,
          "Y": 0.5395888686180115
        },
        {
          "X": 0.33275753259658813,
          "Y": 0.5402967929840088
        },
        {
          "X": 0.3327506184577942,
          "Y": 0.5624536275863647
        },
        {
          "X": 0.1462840586900711,
          "Y": 0.5617520213127136
        }
      ]
    }
  },
  {
    "Text": "018379573482/5311",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.29114848375320435,
        "Height": 0.02177661657333374,
        "Left": 0.5274770259857178,
        "Top": 0.5380648374557495
      },
      "Polygon": [
        {
          "X": 0.5274770259857178,
          "Y": 0.5380648374557495
        },
        {
          "X": 0.8184008002281189,
          "Y": 0.5391712784767151
        },
        {
          "X": 0.8186255097389221,
          "Y": 0.5598414540290833
        },
        {
          "X": 0.5275633335113525,
          "Y": 0.5587441921234131
        }
      ]
    }
  },
  {
    "Text": "Viite:",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.10000422596931458,
        "Height": 0.019036876037716866,
        "Left": 0.14631761610507965,
        "Top": 0.5901595950126648
      },
      "Polygon": [
        {
          "X": 0.1464034467935562,
          "Y": 0.5901595950126648
        },
        {
          "X": 0.24632184207439423,
          "Y": 0.5905309915542603
        },
        {
          "X": 0.24627892673015594,
          "Y": 0.6091964840888977
        },
        {
          "X": 0.14631761610507965,
          "Y": 0.6088279485702515
        }
      ]
    }
  },
  {
    "Text": "211129336196",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.20704959332942963,
        "Height": 0.019655801355838776,
        "Left": 0.6156004667282104,
        "Top": 0.5871403217315674
      },
      "Polygon": [
        {
          "X": 0.6156004667282104,
          "Y": 0.5871403217315674
        },
        {
          "X": 0.8224431872367859,
          "Y": 0.5879107117652893
        },
        {
          "X": 0.8226500749588013,
          "Y": 0.606796145439148
        },
        {
          "X": 0.6157174706459045,
          "Y": 0.6060317158699036
        }
      ]
    }
  },
  {
    "Text": "Debit/Veloitus",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.24387675523757935,
        "Height": 0.022575661540031433,
        "Left": 0.14576397836208344,
        "Top": 0.6375600099563599
      },
      "Polygon": [
        {
          "X": 0.14586389064788818,
          "Y": 0.6375600099563599
        },
        {
          "X": 0.3896191418170929,
          "Y": 0.6384474039077759
        },
        {
          "X": 0.389640748500824,
          "Y": 0.6601356863975525
        },
        {
          "X": 0.14576397836208344,
          "Y": 0.6592563390731812
        }
      ]
    }
  },
  {
    "Text": "14,95",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08228108286857605,
        "Height": 0.02055634744465351,
        "Left": 0.6722087860107422,
        "Top": 0.636845052242279
      },
      "Polygon": [
        {
          "X": 0.6722087860107422,
          "Y": 0.636845052242279
        },
        {
          "X": 0.7542999982833862,
          "Y": 0.6371442675590515
        },
        {
          "X": 0.7544898986816406,
          "Y": 0.6574013829231262
        },
        {
          "X": 0.6723605394363403,
          "Y": 0.6571047306060791
        }
      ]
    }
  },
  {
    "Text": "EUR",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.049477770924568176,
        "Height": 0.018508566543459892,
        "Left": 0.773640513420105,
        "Top": 0.6365290880203247
      },
      "Polygon": [
        {
          "X": 0.773640513420105,
          "Y": 0.6365290880203247
        },
        {
          "X": 0.8229175806045532,
          "Y": 0.6367087960243225
        },
        {
          "X": 0.823118269443512,
          "Y": 0.6550377011299133
        },
        {
          "X": 0.7738205194473267,
          "Y": 0.6548594236373901
        }
      ]
    }
  },
  {
    "Text": "Visa",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06869582086801529,
        "Height": 0.01850813254714012,
        "Left": 0.14645899832248688,
        "Top": 0.6642143130302429
      },
      "Polygon": [
        {
          "X": 0.1465427577495575,
          "Y": 0.6642143130302429
        },
        {
          "X": 0.21515481173992157,
          "Y": 0.6644611358642578
        },
        {
          "X": 0.21509982645511627,
          "Y": 0.6827223896980286
        },
        {
          "X": 0.14645899832248688,
          "Y": 0.6824774742126465
        }
      ]
    }
  },
  {
    "Text": "Contactless",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.19159264862537384,
        "Height": 0.020970063284039497,
        "Left": 0.23368096351623535,
        "Top": 0.6622118949890137
      },
      "Polygon": [
        {
          "X": 0.2337334007024765,
          "Y": 0.6622118949890137
        },
        {
          "X": 0.42523685097694397,
          "Y": 0.6629015803337097
        },
        {
          "X": 0.4252736270427704,
          "Y": 0.6831819415092468
        },
        {
          "X": 0.23368096351623535,
          "Y": 0.6824982762336731
        }
      ]
    }
  },
  {
    "Text": "ALV",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.052466362714767456,
        "Height": 0.01845034956932068,
        "Left": 0.19854851067066193,
        "Top": 0.7384694814682007
      },
      "Polygon": [
        {
          "X": 0.1986103653907776,
          "Y": 0.7384694814682007
        },
        {
          "X": 0.2510148882865906,
          "Y": 0.7386517524719238
        },
        {
          "X": 0.25097498297691345,
          "Y": 0.7569198608398438
        },
        {
          "X": 0.19854851067066193,
          "Y": 0.7567390203475952
        }
      ]
    }
  },
  {
    "Text": "VEROTON",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.12230847775936127,
        "Height": 0.01851937361061573,
        "Left": 0.390312522649765,
        "Top": 0.7369153499603271
      },
      "Polygon": [
        {
          "X": 0.390312522649765,
          "Y": 0.7369153499603271
        },
        {
          "X": 0.51255202293396,
          "Y": 0.7373409867286682
        },
        {
          "X": 0.5126209855079651,
          "Y": 0.7554347515106201
        },
        {
          "X": 0.39033079147338867,
          "Y": 0.7550125122070312
        }
      ]
    }
  },
  {
    "Text": "VERO",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06850404292345047,
        "Height": 0.018061110749840736,
        "Left": 0.5653373003005981,
        "Top": 0.7365347743034363
      },
      "Polygon": [
        {
          "X": 0.5653373003005981,
          "Y": 0.7365347743034363
        },
        {
          "X": 0.6337238550186157,
          "Y": 0.7367729544639587
        },
        {
          "X": 0.633841335773468,
          "Y": 0.7545958757400513
        },
        {
          "X": 0.5654268264770508,
          "Y": 0.7543595433235168
        }
      ]
    }
  },
  {
    "Text": "VEROLLINEN",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.17192544043064117,
        "Height": 0.020111123099923134,
        "Left": 0.6697331666946411,
        "Top": 0.7344713807106018
      },
      "Polygon": [
        {
          "X": 0.6697331666946411,
          "Y": 0.7344713807106018
        },
        {
          "X": 0.8414371013641357,
          "Y": 0.7350701689720154
        },
        {
          "X": 0.8416585922241211,
          "Y": 0.7545825242996216
        },
        {
          "X": 0.6698778867721558,
          "Y": 0.7539888620376587
        }
      ]
    }
  },
  {
    "Text": "24.00%",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.10484228283166885,
        "Height": 0.019306134432554245,
        "Left": 0.16352033615112305,
        "Top": 0.7628084421157837
      },
      "Polygon": [
        {
          "X": 0.1635996550321579,
          "Y": 0.7628084421157837
        },
        {
          "X": 0.2683626413345337,
          "Y": 0.7631686925888062
        },
        {
          "X": 0.26832881569862366,
          "Y": 0.782114565372467
        },
        {
          "X": 0.16352033615112305,
          "Y": 0.7817573547363281
        }
      ]
    }
  },
  {
    "Text": "12.06",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08355052769184113,
        "Height": 0.018875233829021454,
        "Left": 0.41117748618125916,
        "Top": 0.7617074251174927
      },
      "Polygon": [
        {
          "X": 0.41117748618125916,
          "Y": 0.7617074251174927
        },
        {
          "X": 0.4946648180484772,
          "Y": 0.7619947791099548
        },
        {
          "X": 0.4947279989719391,
          "Y": 0.7805826663970947
        },
        {
          "X": 0.41120511293411255,
          "Y": 0.7802976965904236
        }
      ]
    }
  },
  {
    "Text": "2.89",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.06900174915790558,
        "Height": 0.018940677866339684,
        "Left": 0.5825802087783813,
        "Top": 0.7608566284179688
      },
      "Polygon": [
        {
          "X": 0.5825802087783813,
          "Y": 0.7608566284179688
        },
        {
          "X": 0.6514511704444885,
          "Y": 0.76109379529953
        },
        {
          "X": 0.6515820026397705,
          "Y": 0.7797972559928894
        },
        {
          "X": 0.5826814770698547,
          "Y": 0.7795620560646057
        }
      ]
    }
  },
  {
    "Text": "14.95",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08246298879384995,
        "Height": 0.01920139044523239,
        "Left": 0.7596209049224854,
        "Top": 0.7593171000480652
      },
      "Polygon": [
        {
          "X": 0.7596209049224854,
          "Y": 0.7593171000480652
        },
        {
          "X": 0.8418691158294678,
          "Y": 0.7596006989479065
        },
        {
          "X": 0.8420839309692383,
          "Y": 0.7785184979438782
        },
        {
          "X": 0.759800136089325,
          "Y": 0.7782372832298279
        }
      ]
    }
  },
  {
    "Text": "YHTEENSA",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.14016325771808624,
        "Height": 0.020393401384353638,
        "Left": 0.14593923091888428,
        "Top": 0.7869476079940796
      },
      "Polygon": [
        {
          "X": 0.14603058993816376,
          "Y": 0.7869476079940796
        },
        {
          "X": 0.2861025035381317,
          "Y": 0.787423849105835
        },
        {
          "X": 0.28607505559921265,
          "Y": 0.8073410391807556
        },
        {
          "X": 0.14593923091888428,
          "Y": 0.8068690896034241
        }
      ]
    }
  },
  {
    "Text": "12.06",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08346918225288391,
        "Height": 0.018890228122472763,
        "Left": 0.41165778040885925,
        "Top": 0.7861757874488831
      },
      "Polygon": [
        {
          "X": 0.41165778040885925,
          "Y": 0.7861757874488831
        },
        {
          "X": 0.49506354331970215,
          "Y": 0.7864595651626587
        },
        {
          "X": 0.49512696266174316,
          "Y": 0.8050659894943237
        },
        {
          "X": 0.41168561577796936,
          "Y": 0.8047845959663391
        }
      ]
    }
  },
  {
    "Text": "2.89",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.0701785534620285,
        "Height": 0.01954718865454197,
        "Left": 0.582205593585968,
        "Top": 0.7851496934890747
      },
      "Polygon": [
        {
          "X": 0.582205593585968,
          "Y": 0.7851496934890747
        },
        {
          "X": 0.6522488594055176,
          "Y": 0.7853882312774658
        },
        {
          "X": 0.6523841619491577,
          "Y": 0.8046968579292297
        },
        {
          "X": 0.582309901714325,
          "Y": 0.8044604659080505
        }
      ]
    }
  },
  {
    "Text": "14.95",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.08309939503669739,
        "Height": 0.019619278609752655,
        "Left": 0.7593206763267517,
        "Top": 0.7835712432861328
      },
      "Polygon": [
        {
          "X": 0.7593206763267517,
          "Y": 0.7835712432861328
        },
        {
          "X": 0.842200517654419,
          "Y": 0.7838537693023682
        },
        {
          "X": 0.8424201011657715,
          "Y": 0.8031905293464661
        },
        {
          "X": 0.7595036029815674,
          "Y": 0.8029104471206665
        }
      ]
    }
  },
  {
    "Text": "Käyttämättomillä",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.27785271406173706,
        "Height": 0.02519434504210949,
        "Left": 0.1817845106124878,
        "Top": 0.8337692022323608
      },
      "Polygon": [
        {
          "X": 0.1818758249282837,
          "Y": 0.8337692022323608
        },
        {
          "X": 0.45957431197166443,
          "Y": 0.8346924185752869
        },
        {
          "X": 0.45963722467422485,
          "Y": 0.8589635491371155
        },
        {
          "X": 0.1817845106124878,
          "Y": 0.8580506443977356
        }
      ]
    }
  },
  {
    "Text": "ja",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.03213497996330261,
        "Height": 0.020716877654194832,
        "Left": 0.48000720143318176,
        "Top": 0.8358924388885498
      },
      "Polygon": [
        {
          "X": 0.48000720143318176,
          "Y": 0.8358924388885498
        },
        {
          "X": 0.5120640397071838,
          "Y": 0.8359989523887634
        },
        {
          "X": 0.5121421813964844,
          "Y": 0.8566092848777771
        },
        {
          "X": 0.4800702631473541,
          "Y": 0.8565037846565247
        }
      ]
    }
  },
  {
    "Text": "myyntikuntoisilla",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.29655787348747253,
        "Height": 0.025761803612113,
        "Left": 0.528278112411499,
        "Top": 0.8312723636627197
      },
      "Polygon": [
        {
          "X": 0.528278112411499,
          "Y": 0.8312723636627197
        },
        {
          "X": 0.8245649337768555,
          "Y": 0.8322591185569763
        },
        {
          "X": 0.8248359560966492,
          "Y": 0.8570341467857361
        },
        {
          "X": 0.5283812880516052,
          "Y": 0.856058657169342
        }
      ]
    }
  },
  {
    "Text": "tuotteilla",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.173092782497406,
        "Height": 0.02104528807103634,
        "Left": 0.234453022480011,
        "Top": 0.8600364923477173
      },
      "Polygon": [
        {
          "X": 0.23450537025928497,
          "Y": 0.8600364923477173
        },
        {
          "X": 0.4075171649456024,
          "Y": 0.8606044054031372
        },
        {
          "X": 0.407545804977417,
          "Y": 0.881081759929657
        },
        {
          "X": 0.234453022480011,
          "Y": 0.8805192708969116
        }
      ]
    }
  },
  {
    "Text": "vaihto-",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.12051359564065933,
        "Height": 0.02016407437622547,
        "Left": 0.42581090331077576,
        "Top": 0.8601813316345215
      },
      "Polygon": [
        {
          "X": 0.42581090331077576,
          "Y": 0.8601813316345215
        },
        {
          "X": 0.546234130859375,
          "Y": 0.8605767488479614
        },
        {
          "X": 0.5463244915008545,
          "Y": 0.8803454041481018
        },
        {
          "X": 0.4258468747138977,
          "Y": 0.8799536824226379
        }
      ]
    }
  },
  {
    "Text": "ja",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.03227944299578667,
        "Height": 0.020812874659895897,
        "Left": 0.5674953460693359,
        "Top": 0.8600784540176392
      },
      "Polygon": [
        {
          "X": 0.5674953460693359,
          "Y": 0.8600784540176392
        },
        {
          "X": 0.5996548533439636,
          "Y": 0.8601840734481812
        },
        {
          "X": 0.5997747778892517,
          "Y": 0.8808913230895996
        },
        {
          "X": 0.5676000714302063,
          "Y": 0.8807867169380188
        }
      ]
    }
  },
  {
    "Text": "palautus-",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.15480534732341766,
        "Height": 0.02321711555123329,
        "Left": 0.617091178894043,
        "Top": 0.8583235740661621
      },
      "Polygon": [
        {
          "X": 0.617091178894043,
          "Y": 0.8583235740661621
        },
        {
          "X": 0.7716756463050842,
          "Y": 0.8588317632675171
        },
        {
          "X": 0.771896481513977,
          "Y": 0.8815407156944275
        },
        {
          "X": 0.6172317862510681,
          "Y": 0.8810378909111023
        }
      ]
    }
  },
  {
    "Text": "oikeus",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.10396437346935272,
        "Height": 0.019333677366375923,
        "Left": 0.30415403842926025,
        "Top": 0.885866105556488
      },
      "Polygon": [
        {
          "X": 0.3041723072528839,
          "Y": 0.885866105556488
        },
        {
          "X": 0.40809157490730286,
          "Y": 0.8862029314041138
        },
        {
          "X": 0.4081183969974518,
          "Y": 0.9051997661590576
        },
        {
          "X": 0.30415403842926025,
          "Y": 0.9048659801483154
        }
      ]
    }
  },
  {
    "Text": "6.1.2022",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.1421462446451187,
        "Height": 0.019888486713171005,
        "Left": 0.4238494634628296,
        "Top": 0.884508490562439
      },
      "Polygon": [
        {
          "X": 0.4238494634628296,
          "Y": 0.884508490562439
        },
        {
          "X": 0.5658982396125793,
          "Y": 0.8849693536758423
        },
        {
          "X": 0.5659956932067871,
          "Y": 0.9043970108032227
        },
        {
          "X": 0.4238838851451874,
          "Y": 0.9039403796195984
        }
      ]
    }
  },
  {
    "Text": "saakka",
    "Geometry": {
      "BoundingBox": {
        "Width": 0.1024591401219368,
        "Height": 0.01937749981880188,
        "Left": 0.5838930010795593,
        "Top": 0.8837540149688721
      },
      "Polygon": [
        {
          "X": 0.5838930010795593,
          "Y": 0.8837540149688721
        },
        {
          "X": 0.6862042546272278,
          "Y": 0.8840861320495605
        },
        {
          "X": 0.6863521337509155,
          "Y": 0.9031314849853516
        },
        {
          "X": 0.5839964151382446,
          "Y": 0.9028023481369019
        }
      ]
    }
  }
]

In [ ]:
bytes_image = base64.b64decode(base64_image.encode('ascii'))
image = Image.open(io.BytesIO(bytes_image))
image

In [12]:
class ReceiptDataset(Dataset):
    def __init__(self, sample_loaders, processor, label_to_index):
        self.sample_loaders = sample_loaders
        self.processor = processor
        self.label_to_index = label_to_index

        # init the sample IDs to an array of (sample_loader_idx, List[string]) tuples
        self.sample_ids = []

        for i, sample_loader in enumerate(sample_loaders):
            ids = sample_loader.ids()
            self.sample_ids.append((i, ids))

    def __len__(self):
        total = 0

        for _, ids in self.sample_ids:
            total += len(ids)

        return total

    def __getitem__(self, idx):
        encoding = self.encoding(idx)
        # FIXME: the batch dimension should be removed by passing the `prepend_batch_axis` as `False`
        return { key: value.squeeze() for key, value in encoding.items() }

    def encoding(self, idx):
        # find the correct (sample_loader, sample_id) tuple
        sample_cum_sum = 0

        sample_loader = None
        sample_id = None

        for i, ids in self.sample_ids:
            total_cum_sum = sample_cum_sum + len(ids)

            if idx < total_cum_sum:
                # this sample loader is the one for the index
                sample_loader = self.sample_loaders[i]
                sample_id = ids[idx - sample_cum_sum]

                break
            
            sample_cum_sum = total_cum_sum
            
        if sample_loader is None:
            raise "sample loader not found"

        # retrieve the sample and run it through the processor
        sample = sample_loader.sample(sample_id, with_image=True)

        image = sample["image"]
        width, height = image.size

        words = sample["words"]
        boxes = [self._normalize_bbox(bbox, width, height) for bbox in sample["boxes"]]
        word_labels = [self.label_to_index(label) for label in sample["labels"]]

        return processor(image, words, boxes=boxes, word_labels=word_labels,
            # we add padding (and truncate) to the max length to be able to use batches
            max_length=512,

            truncation=True,
            padding=transformers.file_utils.PaddingStrategy.MAX_LENGTH,
            
            # the input is already split into words and not a full sequence of text
            is_split_into_words=True,
            
            # we want pytorch tensors back
            return_tensors="pt",
        )

    def _normalize_bbox(self, bbox, width, height):
        # normalize every bounding box to the range of 0-1000
        return [
            int(1000 * (bbox[0] / width)),
            int(1000 * (bbox[1] / height)),
            int(1000 * (bbox[2] / width)),
            int(1000 * (bbox[3] / height)),
        ]

In [13]:
labels = ["ADDRESS", "COMPANY", "DATE", "TOTAL", "TOTAL_TAX_24", "TOTAL_TAX_14", "TOTAL_TAX_10"]

label_to_index = { "O": 0 }

for i, label in enumerate(labels):
    label_to_index[f"B-{label}"] = 2 * i + 1
    label_to_index[f"I-{label}"] = 2 * (i + 1)

index_to_label = { index: label for label, index in label_to_index.items() }

label_to_index_fn = lambda label: label_to_index[label]
index_to_label_fn = lambda idx: index_to_label[idx]

In [14]:
"""
    "Geometry": {
      "BoundingBox": {
        "Width": 0.13503558933734894,
        "Height": 0.019257284700870514,
        "Left": 0.33889734745025635,
        "Top": 0.17324066162109375
      }
"""

class CustomSampleLoader:
    def __init__(self, image, blocks):
        self.image = image
        self.blocks = blocks

    def ids(self):
        return ["1"]

    def sample(self, id, with_image=False):
        words = list(map(lambda x: x["Text"], self.blocks))
        size = self.image.size

        boxes = []

        for block in self.blocks:
            bbox = block["Geometry"]["BoundingBox"]
            x, y, width, height = bbox["Left"], bbox["Top"], bbox["Width"], bbox["Height"]

            # convert the coordinates to absolute values to be usable by the processor
            boxes.append((
                x * size[0],
                y * size[1],
                (x + width) * size[0],
                (y + height) * size[1],
            ))

        result = { "words": words, "boxes": boxes, "labels": ["O"] * len(boxes) }

        if with_image:
            result["image"] = self.image

        return result

In [15]:
sample_loader = CustomSampleLoader(image, blocks)
print(f"{len(sample_loader.ids())} samples")

1 samples


In [ ]:
"""
DO NOT INCLUDE IN THE INFERENCE SCRIPT!!!
"""

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def display_sample(image, boxes, labels, show_no_label=False):
    fig, ax = plt.subplots(figsize=(16, 16))

    ax.imshow(image)

    for i, box in enumerate(boxes):
        if labels[i] == "O" and not show_no_label:
            continue

        x0, y0, x2, y2 = box

        color_map = {
            "o": "b",
            "date": "m",
            "company": "r",
            "address": "orange",
            "total": "g",
            "total_tax_24": "slateblue",
            "total_tax_14": "lightseagreen",
            "total_tax_10": "teal",
        }

        label_name = labels[i].split("-")[-1].lower()

        rect = patches.Rectangle((x0, y0), x2 - x0, y2 - y0,
            fill=False,
            linewidth=1.5,
            edgecolor=color_map[label_name],
        )

        ax.add_patch(rect)

    plt.show()
    
sample = sample_loader.sample(sample_loader.ids()[0], with_image=True)

boxes, labels, image = sample['boxes'], sample['labels'], sample['image']

display_sample(image, boxes, labels, show_no_label=True)
print("")

In [17]:
from transformers import LayoutLMv2ForTokenClassification, LayoutXLMTokenizerFast, LayoutLMv2FeatureExtractor, LayoutXLMProcessor

In [18]:
num_labels = len(label_to_index.keys())

In [19]:
tokenizer = LayoutXLMTokenizerFast.from_pretrained("microsoft/layoutxlm-base")

In [20]:
model = LayoutLMv2ForTokenClassification.from_pretrained("./inference-model", num_labels=num_labels)

In [21]:
processor = LayoutXLMProcessor(LayoutLMv2FeatureExtractor(apply_ocr=False), tokenizer)

In [22]:
receipt_dataset = ReceiptDataset([CustomSampleLoader(image, blocks)], processor, label_to_index_fn)
receipt_sample = receipt_dataset[0]

print(f"Total number of samples: {len(receipt_dataset)}\n")

for key in receipt_sample.keys():
    print(f"{key}: {receipt_sample[key].shape}")

Total number of samples: 1

input_ids: torch.Size([512])
attention_mask: torch.Size([512])
bbox: torch.Size([512, 4])
labels: torch.Size([512])
image: torch.Size([3, 224, 224])


In [21]:
receipt_dataloader = DataLoader(receipt_dataset, batch_size=1)
receipt_batch = next(iter(receipt_dataloader))

In [22]:
model.eval()

with torch.no_grad():
    receipt_batch_out = model(**receipt_batch)

In [38]:
def tokens_to_entity_candidates(tokens, labels):
    # TODO: it would probably be worth to explore to assign entities at the word-level and not at the token-level.
    #       now, it's possible that the entities contain partial words which is of course not ideal.
    
    entity_names = ["company", "address", "date", "total", "total_tax_24", "total_tax_14", "total_tax_10"]
    entity_candidates = { name: [] for name in entity_names }

    context = None

    for i, token in enumerate(tokens):
        label = labels[i]                           # e.g. B-COMPANY
        entity_name = label.split("-")[-1].lower()  # e.g. company

        if label == "O" or entity_name not in entity_names:
            context = None
            continue

        is_start = label.startswith("B-")

        if is_start:
            # if we are instantly continuing from the previous segment (i.e. B-X, I-X, B-X, I-X, ...)
            is_same_segment = context == entity_name
            
            if is_same_segment:
                # we will consider these intra-segment B- labels as if they were I-
                entity_candidates[entity_name][-1].append(token)
            else:
                entity_candidates[entity_name].append([token])

            context = entity_name
            continue

        if context == entity_name:
            # we will continue the last segment
            entity_candidates[entity_name][-1].append(token)
        else:
            # otherwise, we just started a new segment with a non-start label... ignore (at least for now)
            context = None

    return entity_candidates

In [39]:
def select_longest_candidate(candidates):
    result = ""

    for candidate in candidates:
        if len(candidate) > len(result):
            result = candidate

    return result

In [40]:
def select_date_candidate(candidates):
    date_regex = "(\d{1,4}[\-\./]\d{1,4}[\-\./]\d{1,4})"

    for candidate in candidates:
        result = re.search(date_regex, candidate)
        
        if result:
            return result.group(0)
    
    return select_longest_candidate(candidates)

In [41]:
def select_amount_candidate(candidates):
    date_regex = "(\d+[\.,]\d{1,2})"

    for candidate in candidates:
        result = re.search(date_regex, candidate)
        
        if result:
            return result.group(0)
    
    return select_longest_candidate(candidates)

In [42]:
def select_candidate(entity, candidates):
    if len(candidates) == 0:
        return ""

    if entity == "date":
        return select_date_candidate(candidates)
    
    if entity.startswith("total"):
        return select_amount_candidate(candidates)
    
    return select_longest_candidate(candidates)

In [43]:
def prediction_to_entities(encoding, prediction):
    # convert the prediction to a list of label strings
    pred_labels = prediction.argmax(axis=-1).tolist()
    pred_labels = [index_to_label[idx] for idx in pred_labels]
    
    # we can now extract the entity candidates
    tokens = encoding['input_ids'][0].tolist()
    candidates = tokens_to_entity_candidates(tokens, pred_labels)
    
    # convert the candidate input ID sequences to text
    for key in candidates.keys():
        for i in range(len(candidates[key])):
            candidates[key][i] = tokenizer.decode(candidates[key][i], skip_special_tokens=True)

    # select the best candidate for every entity (and clean up the predictions)
    candidates = { key: select_candidate(key, candidates[key]).strip() for key in candidates.keys() }
    
    return candidates

In [44]:
receipt_batch_out

TokenClassifierOutput(loss=tensor(0.5993), logits=tensor([[[ 9.2131e-01,  1.6642e-01, -6.4350e-01,  ..., -8.1245e-01,
          -1.1169e+00, -9.5972e-01],
         [ 7.4447e-01,  1.2866e+00, -1.2439e+00,  ..., -5.7488e-01,
          -4.1540e-02, -8.0112e-01],
         [ 7.6168e-01, -5.1954e-02, -9.1858e-01,  ..., -8.6875e-01,
          -1.0535e+00, -8.8863e-01],
         ...,
         [ 6.3614e+00, -7.6952e-02,  1.3486e-01,  ..., -7.5291e-01,
          -1.2048e+00, -1.2162e+00],
         [ 6.3716e+00, -8.5498e-02,  9.5150e-02,  ..., -7.7814e-01,
          -1.1779e+00, -1.1765e+00],
         [ 5.7779e+00,  6.5287e-04,  3.4786e-03,  ..., -7.9890e-01,
          -8.8389e-01, -1.1369e+00]]]), hidden_states=None, attentions=None)

In [45]:
receipt_batch_out.logits.squeeze().shape

torch.Size([512, 15])

In [46]:
encoding = receipt_dataset.encoding(0)
prediction = receipt_batch_out.logits.squeeze()

entities = prediction_to_entities(encoding, prediction)

In [47]:
entities

{'company': 'Sokos Helsinki',
 'address': '',
 'date': '29-11-2021',
 'total': '14.95',
 'total_tax_24': '',
 'total_tax_14': '14.95',
 'total_tax_10': ''}

In [26]:
import inference

In [27]:
import importlib
importlib.reload(inference)

<module 'inference' from '/home/ec2-user/SageMaker/notebook-2022-10-07/inference.py'>

In [28]:
inference.predict_fn({ "inputs": { "base64_image": base64_image, "blocks": blocks } }, (model, tokenizer))

{'result': {'company': 'Sokos Helsinki',
  'address': '',
  'date': '29-11-2021',
  'total': '14.95',
  'total_tax_24': '',
  'total_tax_14': '14.95',
  'total_tax_10': ''}}